# Oxygen fugacity calculator with variable spinel compositions
## Solution models from the MELTS data/model collection

In [43]:
import pymelts
import numpy as np

### Create instance of the MELTS engine class
This will generate lists of oxide composition variables and a list of permitted phase names

In [44]:
engine = pymelts.Engine()
print engine.get_oxide_names()
print engine.get_phase_names()

['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'NiO', 'CoO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'H2O', 'CO2', 'SO3', 'Cl2O-1', 'F2O-1']
['olivine', 'fayalite', 'sphene', 'garnet', 'melilite', 'orthopyroxene', 'clinopyroxene', 'aegirine', 'aenigmatite', 'cummingtonite', 'amphibole', 'hornblende', 'biotite', 'muscovite', 'feldspar', 'quartz', 'tridymite', 'cristobalite', 'nepheline', 'kalsilite', 'leucite', 'corundum', 'sillimanite', 'rutile', 'perovskite', 'spinel', 'rhm-oxide', 'ortho-oxide', 'whitlockite', 'apatite', 'water', 'fluid', 'alloy-solid', 'alloy-liquid', 'lime', 'periclase', 'calcite', 'aragonite', 'magnesite', 'siderite', 'dolomite', 'spurrite', 'tilleyite', 'diamond', 'graphite', 'system', 'liquid']


## Gordon's spinel compositions 
```
Sample No	N	Al2O3	FeO	SiO2	MgO	Total  
						
AHPC-19	8	 3.91	88.37	0.05	0.43	92.76    						
AHPC-21	9	 2.02	90.55	0.02	0.59	93.18  
AHPC-23	7	 6.87	85.11	0.02	1.12	93.12  
AHPC-24	7	 5.42	87.21	0.03	0.71	93.37  
AHPC-25	5	10.78	79.56	0.05	2.55	92.94  
AHPC-26	4	 4.21	88.25	0.07	0.31	91.73						
AHPC-29	5	13.56	74.77	0.03	4.20	92.56  
AHPC-30	8	10.18	79.23	0.02	3.66	93.08  
AHPC-31	4	 5.55	87.20	0.02	0.54	93.31  
```

### Instantiate an instance of a phase class for rhm-oxide and pure magnetite

In [45]:
hematite = pymelts.Phase('rhm-oxide', composition=pymelts.Composition(Fe2O3=159.69))
magnetite = pymelts.Phase('spinel', composition=pymelts.Composition(FeO=71.844, Fe2O3=159.69))

### Properties of oxygen gas (1 bar standard state)
#### Code taken from Gibbs.c in the MELTS package

In [46]:
def oxygen(t, p):
    r = 8.3143
    pr = 1.0
    tr = 298.15
    hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) 
    hs = hs - 1762835.0*(1.0/t-1.0/tr) 
    hs = hs - 18172.91960*np.log(t/tr) 
    hs = hs + 0.5*0.002676*(t*t-tr*tr)
    ss = 205.15 
    ss = ss + 23.10248*np.log(t/tr) 
    ss = ss - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) 
    ss = ss - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) 
    ss = ss + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr)
    return hs - t*ss

### Calculate proterties of the pure hematite, magnetite and oxygen phase at specified T (K) and P (bars)

In [47]:
resHem = hematite.get_properties(temperature=1125.0+273.15, pressure=5000.0)
resMag = magnetite.get_properties(temperature=1125.0+273.15, pressure=5000.0)
mu0O2 = oxygen(1125.0+273.15, 5000.0)
print 'chemical potential of pure hematite = ', resHem['hematite']
print 'chemical potential of pure oxygen = ', mu0O2
print 'chemical potential of pure magnetite', resMag['magnetite']
idealLogfO2 = (6.0*resHem['hematite']-4.0*resMag['magnetite']-mu0O2)/8.3143/(1125+273.15)/np.log(10.0)
print 'theoretical log fO2 from pure phases = ', idealLogfO2

chemical potential of pure hematite =  -1072910.95378
chemical potential of pure oxygen =  -320325.964163
chemical potential of pure magnetite -1505145.60575
theoretical log fO2 from pure phases =  -3.60736532398


### Now, process the spinels
6Fe2O3 = 4Fe3O4 + O2

In [48]:
spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=3.91, FeO=88.37, SiO2=0.05, MgO=0.43))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-19 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=2.02, FeO=90.55, SiO2=0.02, MgO=0.59))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-21 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2
   
spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=6.87, FeO=85.11, SiO2=0.02, MgO=1.12))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-23 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=5.42, FeO=87.21, SiO2=0.03, MgO=0.71))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-24 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=10.78, FeO=79.56, SiO2=0.05, MgO=2.55))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-25 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=4.21, FeO=88.25, SiO2=0.07, MgO=0.31))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-26 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=13.56, FeO=74.77, SiO2=0.03, MgO=4.20))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-29 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=10.18, FeO=79.23, SiO2=0.02, MgO=3.66))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-30 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

spinel = pymelts.Phase('spinel', composition=pymelts.Composition(Al2O3=5.55, FeO=87.20, SiO2=0.02, MgO=0.54))
results = spinel.get_properties(temperature=1125.0+273.15, pressure=5000.0)
logfO2 = (6.0*resHem['hematite'] - 4.0*results['magnetite']-mu0O2)/8.3143/(1125.0+273.15)/np.log(10.0)
print 'AHPC-31 log fO2 = ', logfO2, 'difference (actual-pure)', logfO2 - idealLogfO2

AHPC-19 log fO2 =  -3.28450180788 difference (actual-pure) 0.322863516096
AHPC-21 log fO2 =  -3.40014171957 difference (actual-pure) 0.207223604406
AHPC-23 log fO2 =  -3.05276860505 difference (actual-pure) 0.554596718928
AHPC-24 log fO2 =  -3.16873649627 difference (actual-pure) 0.438628827711
AHPC-25 log fO2 =  -2.77040596386 difference (actual-pure) 0.836959360118
AHPC-26 log fO2 =  -3.27541227417 difference (actual-pure) 0.331953049805
AHPC-29 log fO2 =  -2.58103888944 difference (actual-pure) 1.02632643454
AHPC-30 log fO2 =  -2.72724128151 difference (actual-pure) 0.88012404247
AHPC-31 log fO2 =  -3.17437911791 difference (actual-pure) 0.432986206074
